In [1]:
# loading modules
import ee
import geemap

# initialize ee 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()



In [2]:
# CASE 1: Pakistan Floods 2022
# inputs
# Define the Area of Interest (aoi)
aoi = ee.Geometry.Polygon(
    [[[67.98392410136336, 26.049909335428502],
      [67.98392410136336, 25.42892423506662],
      [68.59778518534773, 25.42892423506662],
      [68.59778518534773, 26.049909335428502]]])


startDate = ee.Date('2022-03-01')
endDate = ee.Date('2022-08-20')
predays = 60
postdays = 20
Map = geemap.Map()
Map.addLayer(aoi, {}, 'Area of Interest')
Map.centerObject(aoi, 10)

In [2]:
# CASE 2: Mumbai
# inputs
# Define the Area of Interest (aoi)
name = 'Mumbai'
fsm_cities_box = ee.FeatureCollection("projects/temporaryproj/assets/fs_cities_boundarybox_combined")
aoi = fsm_cities_box.filter(ee.Filter.eq('name', f'{name}_aoi')).geometry()
fsm_cities = ee.FeatureCollection("projects/temporaryproj/assets/fs_cities_boundary_combined")
aoi_city = fsm_cities.filter(ee.Filter.eq('name', name)).geometry()

startDate = ee.Date('2017-03-01')
endDate = ee.Date('2017-08-22')
predays = 30
postdays = 10
Map = geemap.Map()
Map.addLayer(aoi, {}, 'Area of Interest')
Map.centerObject(aoi)

In [3]:
def get_s1_col(date, days):
  filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    ee.Filter.listContains("transmitterReceiverPolarisation", "VH"),
    ee.Filter.Or(ee.Filter.equals("instrumentMode", "IW"),ee.Filter.equals("instrumentMode", "SM")),
    ee.Filter.bounds(aoi),
    ee.Filter.eq('resolution_meters', 10),
    ee.Filter.date(date, date.advance(days+1, 'day'))
  ]
  
  s1_col = ee.ImageCollection('COPERNICUS/S1_GRD').filter(filters)

  return s1_col

def s1_composite(get_s1_col):
  composite = ee.Image.cat([
  s1_col.select('VH').mean(),
  s1_col.select('VV').mean(),
  s1_col.select('VH').mean()
  ])
    
  return composite.clip(aoi)

s1_pre = get_s1_col(startDate, predays)
s1_post = get_s1_col(endDate, postdays)

# Function to calculate z-score
def calc_zscore(s1_pre, s1_post, direction):
    base_mean = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)).mean()

    anom = s1_post.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .mean().subtract(base_mean).set({'system:time_start': s1_post.get('system:time_start')})

    base_sd = s1_pre.filter(ee.Filter.equals('orbitProperties_pass', direction)) \
        .reduce(ee.Reducer.stdDev()).rename(['VV', 'VH', 'angle'])

    return anom.divide(base_sd).set({'system:time_start': anom.get('system:time_start')})


# Define z-score calculation for DESCENDING and ASCENDING orbits
zscore_des = calc_zscore(s1_pre, s1_post,'DESCENDING')
zscore_asc = calc_zscore(s1_pre, s1_post,'ASCENDING')

# Calculate mean of z-scores for DESCENDING and ASCENDING orbits
zscore = ee.ImageCollection.fromImages([zscore_des, zscore_asc]).mean().clip(aoi)

# Flood mask function
def mapFloods(z, zvv_thd=None, zvh_thd=None, pow_thd=75, elev_thd=800, slp_thd=15):
    if zvv_thd is None:
        zvv_thd = -3
    if zvh_thd is None:
        zvh_thd = -3

    # JRC water mask
    jrc = ee.ImageCollection("JRC/GSW1_1/MonthlyHistory").filterDate('2016-01-01', '2022-01-01')
    jrcvalid = jrc.map(lambda x: x.gt(0)).sum()
    jrcwat = jrc.map(lambda x: x.eq(2)).sum().divide(jrcvalid).multiply(100)
    jrcmask = jrcvalid.gt(0)
    ow = jrcwat.gte(ee.Image(pow_thd))

    # Add elevation and slope masking using fabdem
    elevation = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic() \
        .setDefaultProjection('EPSG:3857', None, 30)
    slope = ee.Terrain.slope(elevation)

    # Classify floods
    vvflag = z.select('VV').lte(ee.Image(zvv_thd))
    vhflag = z.select('VH').lte(ee.Image(zvh_thd))

    flood_class = ee.Image(0).add(vvflag) \
        .add(vhflag.multiply(2)) \
        .where(ow.eq(1), 4) \
        .rename('flood_class') \
        .where(elevation.gt(elev_thd).multiply(ow.neq(1)), 0) \
        .where(slope.gt(slp_thd).multiply(ow.neq(1)), 0)
    # add class 1,2,3 from flood_class and create a new layer with single band
    flood_layer = flood_class.where(flood_class.eq(1), 1) \
        .where(flood_class.eq(2), 1) \
        .where(flood_class.eq(3), 1) \
        .where(flood_class.eq(4), 2) \
        .where(jrcmask.eq(0), 2) \
        .where(flood_class.eq(0), 2) \
        .selfMask() \
        .rename('label')
    return flood_class.clip(aoi), flood_layer.clip(aoi)

# Mask z-score threshold
flood_class, flood_layer = mapFloods(zscore)

In [4]:
Map.addLayer(zscore.select('VV'), {'min': -7, 'max': 7, 'palette': ['red', 'white', 'blue']}, 'ZScore', False)
Map.addLayer(flood_class, {'min': 0, 'max': 4, 'palette': ['#E3E3E3','#FFB100', '#FFB100', '#3E9DFF','#031DC9']}, 'flood classes', False)
Map.addLayer(flood_layer, {'min': 1, 'max': 2, 'palette': ['blue', 'white']}, 'flood layer')

Map

Map(center=[19.073663071484283, 72.87546088454904], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
geemap.ee_export_image_to_drive(flood_layer, description='flood_layer', folder='s1_post', region=aoi, scale=10)

In [4]:

# Some parts of the original code are missing, but the following sections are provided for completeness.

# Create the 'sample' feature collection
bands = ['VV', 'VH']
image = s1_post.select(bands).mean()
field = flood_layer

sample = field.stratifiedSample(
    numPoints=1000,
    classBand='label',
    region=aoi_city,
    scale=10,
    seed=5,
    tileScale=1.5,
    geometries=True
)



In [5]:
def updateFeature(feature):
    value = feature.get('label')
    updated_value = ee.Algorithms.If(value==ee.Number(2), ee.Number(0), value)
    feature = feature.set('label', updated_value)
    return feature
sample_new = sample.map(updateFeature)
label = ee.FeatureCollection(sample_new)
Map.addLayer(label, {}, 'sample')
Map

Map(center=[19.073663071484283, 72.87546088454904], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# change the values in labels so that values if 2 are changed with 0
Map.addLayer(sample1, {}, 'sample')
Map.addLayer(aoi_city, {'color':'red'}, 'aoi_city', False)
Map
# Continue with training and classification as shown in the original code
# ... (code for training and classification continues here)


In [38]:
# export samples as shapefile to gdrive
geemap.ee_to_shp(sample_new, filename='../data/samples_test2.shp')

Generating URL ...
Please wait ...
Data downloaded to e:\current_proj\FMS\codes\data\samples_test2.shp
